In [1]:
!pip install sentence-transformers
!pip install openai

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 5.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 75.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 56.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 26.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 94.7 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125926 sha256=ac4d43079c5a6911b16ed173ae38db22b7e9b89dc5b5065e6faca4b9b5f0d0a2
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
from sentence_transformers import SentenceTransformer, util

#chosen model
model_identifier = "pritamdeka/S-Biomed-Roberta-snli-multinli-stsb"


model = SentenceTransformer(model_identifier , device = "cpu")

import numpy as np
def cosine_similarity(array_01, array_02):
    return np.dot(array_01, array_02) / (
        np.linalg.norm(array_01) * np.linalg.norm(array_02)
    )
def get_similarity(query , passage_list , top=5): 
  query_embedding = model.encode(query)
  passage_embedding = model.encode(passage_list)
  similarities = util.cos_sim(query_embedding, passage_embedding).flatten().tolist()
  docs_with_similarities = list(zip(passage_list, similarities))

  sorted_passagess = sorted(docs_with_similarities, key=lambda x: x[1],reverse=True)
            
        
  return sorted_passagess[:top]

In [3]:
import pickle
import json
import torch
import requests 
import openai

In [4]:
with open('/content/drive/MyDrive/FYP/DATA-sets.raw/HealthCareMagic-200k_questions_to_embeddings.pickle' , 'rb') as fileee: 
    questions_to_embeddings_map = pickle.load(fileee)

with open("/content/drive/MyDrive/FYP/DATA-sets.raw/HealthCareMagic-200k_questions_to_answers.json") as questions_to_answers_001 : 
    questions_to_answers = json.load(questions_to_answers_001)


In [5]:
print(len(questions_to_embeddings_map))

206920


In [6]:
embeddings_map = questions_to_embeddings_map
def get_context_from_shots(question, threshold = 0.7) :
    
    global questions_to_embeddings_map 
    questions_list = list(questions_to_embeddings_map)
    query_embeddings = model.encode(question)

    cosine_scores = util.cos_sim(query_embeddings, list(questions_to_embeddings_map.values()))[0]

    top_questions = []
    top_results = torch.topk(cosine_scores, k=4)
    for score, idx in zip(top_results[0], top_results[1]):
        if score > threshold:
            top_questions.append((questions_list[idx] , score))   
        if not top_questions: 
          print(top_results)
          print("Empty top_docs")
        return top_questions

In [7]:
def call_open_ai_api(prompt):
  api_key = "####"

  openai.api_key = api_key

  response = openai.Completion.create(
    model="text-davinci-003",
    prompt= prompt,
    temperature=0.3,
    max_tokens=256,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0
  )

  return response

In [8]:
def prepare_prompt(question , history) : 
  with open("/content/drive/MyDrive/FYP/Prompts/with-shots-as-context.text") as prompt_file_with_context : 
    prompt = prompt_file_with_context.read() 
  global  questions_to_answers 
  context = ""
  if not history : 
    context = get_context_from_shots(question) 

  if context:
     for each in context:  
        prompt += f"###\nUser:{each[0]}\nAssistant:{questions_to_answers[each[0]]}###\n"
  else : 
      prompt += "###NO KNOWLEDGE FROM SIMILAR CHAT HISTORY AVAILABLE###"
  
  if history : 
    for each in history:
      prompt += "\n\nCurrent chat history:\n\n" 
      prompt += f"###\nUser:{each[0]}\nAssistant:{each[1]}###\n"

  prompt += f"###\nUser:{question}\nAssistant:"   
  # print(prompt)
  return prompt




In [9]:
def answer_user_query(question , history) : 
  prompt = prepare_prompt(question , history)
  response = call_open_ai_api(prompt)
  return response["choices"][0]["text"] 



In [10]:
# history = []
# import pprint

# while True : 
#   print("User:") 
#   question = input()# "I feel a back and leg pain in my lower back and upper leg , is it sciatica pain?"
#   Answer_from_Assistant = answer_user_query(question )
#   print("Assistant:")
#   pp = pprint.PrettyPrinter(width=70, compact=True)
#   pp.pprint( Answer_from_Assistant)
#   history.append((question , Answer_from_Assistant ))



In [11]:
#serving as an api 
!pip install flask
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip -o ngrok-stable-linux-amd64.zip

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
--2023-05-17 08:42:53--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 52.202.168.65, 18.205.222.128, 54.161.241.46, ...
Connecting to bin.equinox.io (bin.equinox.io)|52.202.168.65|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13921656 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  13.28M  19.3MB/s    in 0.7s    

2023-05-17 08:42:54 (19.3 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13921656/13921656]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


In [12]:
!pip install pyngrok



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 681.2/681.2 kB 13.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyngrok: filename=pyngrok-6.0.0-py3-none-any.whl size=19867 sha256=7b8656cf2f3d3a54ecf6f5e3abd0bba2b26ee00cecaaa3c3b00df03c89f0a798
  Stored in directory: /root/.cache/pip/wheels/5c/42/78/0c3d438d7f5730451a25f7ac6cbf4391759d22a67576ed7c2c
Successfully built pyngrok


In [13]:
!pip install flask
!pip install flask-ngrok

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [14]:
!ngrok authtoken ########


Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [17]:
!pip install flask_cors

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [18]:
from flask_cors import CORS

In [ ]:
from flask import *
from flask_ngrok import run_with_ngrok
from flask import request  , json , jsonify
app = Flask(__name__)

@app.route('/index')
def home():
  return 'Hello World'

@app.route("/get_response_from_bot" ,  methods=['GET', 'POST'])
def get_response_from_bot() : 
  request_json = request.get_json()
  query = request_json.get("query")
  history = request_json.get("history")
  if not history : 
    histroy = []
  print(query , "\n\n", history )
  response = answer_user_query(query , history) 
  print(response)
  history.append((query,response))
  return jsonify( { 
      "history" : history , "response" : response
  })


CORS(app)
run_with_ngrok(app)
app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://dba9-34-125-141-226.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [17/May/2023 10:25:13] "OPTIONS /get_response_from_bot HTTP/1.1" 200 -


Give me a famous movie as three emojis and I'll guess the title. 

 []
torch.return_types.topk(
values=tensor([0.4043, 0.4015, 0.3973, 0.3961]),
indices=tensor([107775,  42784,  63621,  91266]))
Empty top_docs


...

What to do when I get a severe headache after C section? 

 []


INFO:werkzeug:127.0.0.1 - - [17/May/2023 11:18:17] "POST /get_response_from_bot HTTP/1.1" 200 -


 It is best to consult with a doctor if you are experiencing a severe headache after a C section. It is important to rule out any underlying medical conditions that may be causing the headache. Depending on the cause, your doctor may recommend medications, lifestyle changes, or other treatments to help alleviate the headache.
torch.return_types.topk(
values=tensor([0.6720, 0.6618, 0.6382, 0.6244]),
indices=tensor([203643,   8745, 140056, 180152]))
Empty top_docs


INFO:werkzeug:127.0.0.1 - - [17/May/2023 11:18:19] "POST /get_response_from_bot HTTP/1.1" 200 -


 It is important to have a proper checkup from a doctor if you are experiencing severe headaches after a C section. It could be a sign of a more serious condition. I cannot answer this question with certainty as I have limited knowledge related to this question in my database.


In [ ]:
ngrok.kill()